In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('../data/SilasMarner/character-tagging/SilasMarner.entities', delimiter='\t')
df

,COREF,start_token,end_token,prop,cat,text
0,165,11,12,NOM,FAC,the farmhouses
1,166,16,25,NOM,PER,"great ladies , clothed in silk and thread - lace"
2,166,28,28,PRON,PER,their
3,167,42,47,NOM,LOC,districts far away among the lanes
4,168,46,47,NOM,FAC,the lanes
...,...,...,...,...,...,...
11700,3225,86676,86678,NOM,FAC,a pretty home
11701,2065,86679,86679,PRON,PER,ours
11702,144,86682,86682,PRON,PER,I
11703,443,86684,86684,NOM,PER,nobody


In [3]:
df_characters = df[df.cat == 'PER']
df_characters

,COREF,start_token,end_token,prop,cat,text
1,166,16,25,NOM,PER,"great ladies , clothed in silk and thread - lace"
2,166,28,28,PRON,PER,their
6,170,58,82,NOM,PER,"certain pallid undersized men , who , by the s..."
7,171,69,73,NOM,PER,the brawny country - folk
8,172,84,85,NOM,PER,The shepherd
...,...,...,...,...,...,...
11699,144,86672,86672,PROP,PER,Eppie
11701,2065,86679,86679,PRON,PER,ours
11702,144,86682,86682,PRON,PER,I
11703,443,86684,86684,NOM,PER,nobody


In [4]:
len(df_characters.groupby(df.COREF))

1543

In [5]:
characters = {}

for coref_id, coref_df in df_characters.groupby(df.COREF):
    try:
        proper_noun_df = coref_df[coref_df.prop == 'PROP']
        if not proper_noun_df.empty:
            name = proper_noun_df.text[proper_noun_df.text.str.len() == proper_noun_df.text.str.len().max()].unique()[0]
            characters[coref_id] = name
    except:
        continue

characters

{1: 'the Raveloe',
 4: 'the Rainbow',
 5: 'Tarley',
 10: 'Batherley',
 12: 'Durham',
 15: 'the Warrens',
 32: 'father Silas , who had taken her golden curls for his lost guineas brought back to him',
 33: 'Experience',
 34: 'Jem Rodney',
 35: 'old Squire Cass',
 36: 'the argumentative Mr. Macey',
 37: 'Old Master Marner',
 38: 'Sally Oates',
 39: 'David',
 40: 'Jonathan',
 41: 'William Dane',
 42: 'Satan',
 43: 'God Almighty',
 44: 'Brother',
 45: 'Poor Marner',
 46: 'Christianity',
 48: 'Mrs. Osgood',
 49: 'Thought',
 50: 'Doctor Kimble',
 51: 'Ann Coulter',
 53: 'King Alfred',
 54: 'the Mr. Osgood',
 55: 'the Osgoods',
 56: 'Providence',
 57: 'Betty Jay',
 58: 'Dunstan Cass',
 59: 'Dunsey Cass',
 60: 'King George',
 61: 'the artful Godfrey',
 62: 'Miss Nancy Lammeter',
 63: 'the Miss Lammeters',
 64: 'Fowler',
 65: 'Cox',
 66: 'Molly Farren',
 67: 'Bob Cass',
 68: 'Wildfire',
 69: 'old Bryce',
 70: 'Keating',
 71: 'Snuff',
 72: 'Lord Cromleck',
 73: "Marner's",
 74: 'Miss Priscilla L

In [6]:
from collections import defaultdict
import string


merged_idxs = set()
merged_coref_ids = defaultdict(list)

female_titles = [
    'Mrs.',
    'Lady',
    'Miss',
    'Aunt',
    'Duchess',
]

male_titles = [
    'Sir',
    'Mr.',
    'Lord',
    'Uncle',
    'Colonel',
]

neutral_titles_and_starters = [
    'Dr.',
    'Dear',
    'Poor',
    'Old',
    'Dearsest',
]


# Merge characters based on titles
for i, (coref_id, base_name) in enumerate(characters.items()):
    
    if i in merged_idxs:
        continue
    
    try:
        base_split_name = base_name.split()

        base_titles = []
        base_gender = 'neutral'
        while base_split_name[0] in male_titles + female_titles + neutral_titles_and_starters:

            if base_split_name[0] in male_titles:
                base_gender = 'male'
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            elif base_split_name[0] in female_titles:
                base_gender = 'female'
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            else:
                base_split_name = base_split_name[1:]
                base_titles.append(base_split_name[0])
            
        base_forename = base_split_name[0]
        base_surname = base_split_name[-1]
            

        for j, (other_coref_id, other_name) in enumerate(characters.items()):
            
            if j in merged_idxs:
                continue
            
            if j <= i:
                continue
            other_split_name = other_name.split()
            
            other_titles = []
            other_gender = 'neutral'
            while any([name in male_titles + female_titles + neutral_titles_and_starters for name in other_split_name]):

                if other_split_name[0] in male_titles:
                    other_gender = 'male'
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])
                elif other_split_name[0] in female_titles:
                    other_gender = 'female'
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])
                else:
                    other_split_name = other_split_name[1:]
                    other_titles.append(other_split_name[0])

            other_forename = other_split_name[0]
            other_surname = other_split_name[-1]
            
            if base_gender == 'male' and other_gender == 'female' or base_gender == 'female' and other_gender == 'male':
                continue
            
            if other_forename != other_surname and base_forename != base_surname:
                if base_forename != other_forename or base_surname != other_surname:
                    continue
                else:
                    merged_idxs.add(j)
                    merged_coref_ids[coref_id].append(other_coref_id)
                    print(f'Merging1: {base_name} -> {other_name}')
                    continue
            
            # Otherwise gender is the same/neutral, if the forenames match we should merge
            
            if other_forename == base_forename:
                merged_idxs.add(j)
                merged_coref_ids[coref_id].append(other_coref_id)
                print(f'Merging2: {base_name} -> {other_name}')
    except Exception as e:
        pass

Merging2: Mrs. Osgood -> Miss Osgood
Merging1: Dolly Winthrop -> Dolly Winthrop
Merging2: Dolly Winthrop -> Poor Dolly
Merging2: Solomon Macey -> Old Solomon
Merging2: Miss Ladbrook -> Mrs. Ladbrook


In [7]:
df_merged = df_characters[df_characters['COREF'].isin(list(characters.keys()))]
df_merged.COREF.unique().size

137

In [8]:
for coref_id, merged_coref_list in merged_coref_ids.items():
    for merged_coref_id in merged_coref_list:
        df_merged.loc[df_merged["COREF"] == merged_coref_id, "COREF"] = coref_id
    
df_merged.COREF.unique().size

132

In [9]:
min_mentions = 10

df_filtered = df_merged.groupby('COREF').filter(lambda x: len(x) > min_mentions)
df_filtered.COREF.unique().size

42

In [10]:
df_filtered

,COREF,start_token,end_token,prop,cat,text
48,32,533,534,PROP,PER,Silas Marner
49,32,538,538,PRON,PER,his
55,32,573,573,PROP,PER,Silas
61,32,675,675,PROP,PER,Marner
62,32,683,683,PRON,PER,his
...,...,...,...,...,...,...
11688,32,86599,86599,PROP,PER,Silas
11690,61,86605,86605,PRON,PER,he
11691,144,86607,86607,PROP,PER,Eppie
11699,144,86672,86672,PROP,PER,Eppie


In [11]:
df_filtered['character_name'] = df_filtered.COREF.map(lambda x: characters[x])

In [12]:
df_filtered.to_csv('../data/SilasMarner/processed_characters.csv')

In [13]:
import json


merged_coref_to_new_coref = {}

for coref_id, merged_list in merged_coref_ids.items():
    for merged_id in merged_coref_list:
        merged_coref_to_new_coref[merged_id] = coref_id

with open('../data/SilasMarner/coref_merge_map.json', 'w') as f:
    json.dump(merged_coref_to_new_coref, f)